In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import datetime
import pathlib

import numpy
import os
import sklearn.model_selection

from classifier import classification_data

In [2]:
cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "../features",
    "path_out": "out",
    "test_size": 0.2,
    "train_size": 0.8,
    "random_state": 1234
}

In [3]:
surf = numpy.loadtxt("surf64_iwssip.txt")
n_samples, n_features = surf.shape
x_surf, y_surf = surf[0:, 0:n_features - 1], surf[:, n_features - 1]

kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
kf.get_n_splits(x_surf, y_surf)

5

In [11]:
get_pca = {
    "mobilenetv2": (128, 256, 512, 1024, -1),
    "resnet50v2": (128, 256, 512, 1024, -1),
    "vgg16": (128, 256, 512, -1)
}

path = os.path.join("../features", "unet", "GRAYSCALE", "256", "mobilenetv2", "aa")

import multiprocessing
print(multiprocessing.cpu_count())
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['BLIS_NUM_THREADS'] = '1'
os.environ['MKL_DYNAMIC'] = 'FALSE'

path = "/home/xandao/Documentos/GitHub/features/unet/GRAYSCALE/256/mobilenetv2/aa"
# with sklearn.utils.parallel_backend("threading", n_jobs=multiprocessing.cpu_count()):
    # for i, file in enumerate(sorted(pathlib.Path(os.path.join(cfg["path_base"], "unet", "GRAYSCALE")).rglob("*.npz"))):
    # for i, file in enumerate(sorted(pathlib.Path(os.path.join(cfg["path_base"], "PHOTOSHOP", "GRAYSCALE")).rglob("*.npz"))):
for i, file in enumerate(sorted(pathlib.Path(path).rglob("*.npz"))):
    get_source = list(filter(lambda x: x in str(file).lower(), ["unet", "iwssip", "matlab"]))
    source = get_source[0] if len(get_source) > 0 else None
    get_extractor = list(filter(lambda x: x in str(file).lower(), ["mobilenetv2", "vgg16", "resnet50v2"]))
    extractor = get_extractor[0] if len(get_extractor) > 0 else None
    get_dim = list(filter(lambda x: x in str(file).lower(), ["256", "400", "512"]))
    dim = get_dim[0] if len(get_dim) > 0 else None
    get_color_mode = list(filter(lambda x: x in str(file).lower(), ["rgb", "grayscale"]))
    color_mode = get_color_mode[0] if len(get_color_mode) > 0 else None
    get_orientation = list(filter(lambda x: x in str(file).lower(), ["horizontal", "vertical", "h+v"]))
    orientation = get_orientation[0] if len(get_orientation) > 0 else None
    get_patch = list(filter(lambda x: x in str(file).lower(), ["patch=3", "patch=5", "patch=7"]))
    patch = get_patch[0].split("=")[1] if len(get_patch) > 0 else None

    print(i, len(list(pathlib.Path(path).rglob("*.npz"))), file)
    print(source, extractor, dim, color_mode, orientation, patch)

    data = numpy.load(file)
    dataset = extractor
    n_patch = int(patch)
    x = data["features"]
    y = data["labels"]
    x = numpy.ascontiguousarray(x, dtype=numpy.float32)
    y = numpy.ascontiguousarray(y, dtype=numpy.int32)
    n_samples = x.shape[0]
    n_features = x.shape[1]
    x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

    current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    path = os.path.join(cfg["path_out"], current_datetime, dataset)
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    for pca in get_pca[dataset]:
        x = x_normalized
        if pca > 0:
            x = sklearn.decomposition.PCA(n_components=pca, random_state=cfg["random_state"]).fit_transform(x)

        if pca == -1:
            pca = n_features

        classification_data(cfg, color_mode, dataset, dim, str(file), kf, pca, n_samples, path, x, x_surf, y, n_patch=n_patch, orientation=orientation)
        print("\n")
        break
    data.close()

12
0 3 /home/xandao/Documentos/GitHub/features/unet/GRAYSCALE/256/mobilenetv2/aa/vertical+patch=3+mobilenetv2.npz
unet mobilenetv2 256 grayscale vertical 3
0 DecisionTreeClassifier (900, 128) (225, 128)
(225, 5) (225,)
1 DecisionTreeClassifier (900, 128) (225, 128)
(225, 5) (225,)
2 DecisionTreeClassifier (900, 128) (225, 128)
(225, 5) (225,)
3 DecisionTreeClassifier (900, 128) (225, 128)
(225, 5) (225,)
4 DecisionTreeClassifier (900, 128) (225, 128)
(225, 5) (225,)


1 0 /home/xandao/Documentos/GitHub/features/unet/GRAYSCALE/256/mobilenetv2/aa/vertical+patch=5+mobilenetv2.npz
unet mobilenetv2 256 grayscale vertical 5
0 DecisionTreeClassifier (1500, 128) (375, 128)
(375, 5) (375,)
1 DecisionTreeClassifier (1500, 128) (375, 128)
(375, 5) (375,)
2 DecisionTreeClassifier (1500, 128) (375, 128)
(375, 5) (375,)
3 DecisionTreeClassifier (1500, 128) (375, 128)
(375, 5) (375,)
4 DecisionTreeClassifier (1500, 128) (375, 128)
(375, 5) (375,)


2 0 /home/xandao/Documentos/GitHub/features/unet/GRA